In [4]:
from astropy.io import fits
import numpy as np
import glob
import os
from pathlib import Path
import re
from datetime import datetime

In [226]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [227]:
def check_need_smear(path):
    status = 0
    parts = path.split("st_")
    search_string = f"st_{parts[1]}"
    file_name = "/Users/hattoriakihiro/Documents/after_graduate2/preprocess/check.txt"

    # ファイル全体を一度に読み込む
    with open(file_name, 'r') as file:
        file_content = file.read()
        if search_string in file_content:
            status = 1

    return status

In [228]:
def bias_cor(DAY):
    result = 0

    B0 = 3.18 * pow(10,2)
    B1 = -4.12 * pow(10,-2)
    B2 = 2.00 * pow(10,-5)
    #calculate bias
    result = B0 + (B1 * DAY) + (B2 * pow(DAY,2))

    return result

In [229]:
#smear correction
def smear_cor(data, NV, NH, tEXP, Isky):
    tVCT = 12 * 1024 * 10 ** -6#μs

    result = np.zeros((NV,NH))
    Ismear = np.zeros((NV,NH))

    for H in range (NH):
        Ismear[:,H] = 0
        for V in range (NV):
            #calculate smear
            Ismear[:,H] += (tVCT / (tVCT + tEXP)) * ((data[V,H] - Isky)/1024)
        #smear correction
        result[:,H] =data[:,H] - Ismear[:,H]
        # print(Ismear,data[:,H] , "result",result[:,H] )

    return result

In [236]:
#scattered light correction
def make_coord(x,y):
    a = np.array([[[x,y]]])
    coordinate = np.tile(a,((2*x)-1,(2*y)-1,1))
    for i in range((2*x)-1):
        for j in range((2*y)-1):
            coordinate[i][j]=[i,j]
    return coordinate

def make_base(x,y):
    a = np.array([[[x-1,y-1]]])
    base = np.tile(a,((2*x)-1,(2*y)-1,1))
    return base

def calc_dist(coord,base):
    a = base-coord
    b = np.sum(np.square(a), axis=2)
    c = np.sqrt(b)
    return c

def scattered_light(data, filt):
    ys = data.shape[0]
    xs = data.shape[1]
    result = np.zeros((ys,xs))

    Ai = {"ul":{"1":12.0,"2":8.0,"3":1.2,"4":1.0,"5":0.8,"6":0.7},
      "b":{"1":10.0,"2":1.5,"3":0.3,"4":0.4,"5":0.4,"6":0.5},
      "v":{"1":10.0,"2":1.5,"3":0.3,"4":0.4,"5":0.4,"6":0.5},
      "w":{"1":10.0,"2":1.5,"3":0.6,"4":0.8,"5":0.7,"6":0.6},
      "x":{"1":9.0,"2":3.5,"3":2.0,"4":2.7,"5":2.2,"6":0.5},
      "p":{"1":10.0,"2":5.0,"3":8.3,"4":4.0,"5":6.4,"6":1.8},
      "zs":{"1":50.0,"2":16.0,"3":6.0,"4":9.0,"5":9.5,"6":4.5}
    }
    filter_u=Ai[filt]
    sigma = [8,16,32,64,110,710]
    coord = make_coord(ys,xs)
    base = make_base(ys,xs)
    r = calc_dist(coord, base)
    fufoc = 0
    for k in range(6):
        num = str(k + 1)
        fufoc = fufoc + ((filter_u[num]*np.power(10.,-4))/(np.sqrt(2*np.pi)*sigma[k]))*np.exp(-np.power(r,2)/(2*np.power(sigma[k],2)))
    for y in range(ys):
        for x in range(xs):
            result[y][x]=np.sum(np.multiply(fufoc[(ys-1)-y:(2*ys-1)-y,(xs-1)-x:(2*xs-1)-x],data))
    return data-result

In [231]:
#flat correction
def get_flat(filt, num_bin):
    result = 0
    home_dir = os.path.expanduser("~")
    p = os.path.join(home_dir, "Documents/after_graduate2/preprocess/flat_filt")
    for f in glob.glob(p+"/bin_"+str(num_bin)+"flat_"+filt+".fit"):
        result = fits.open(f)
        result = result[0].data
    return result
def flat_cor(filt, data, num_bin, startH, startV):
    result = 0
    flat_im = get_flat(filt, num_bin)
    startV = startV // num_bin
    startH = startH // num_bin
    if filt =="p" or filt == "w":
        result = data[startV : startV + data.shape[0], startH : startH + data.shape[1]] / flat_im[startV : startV + data.shape[0], startH : startH + data.shape[1]]
    elif filt == "b":
        result = data[startV : startV + data.shape[0],startH + 70//num_bin : startH + data.shape[1]] / flat_im[startV : startV + data.shape[0],startH + 70//num_bin : startH + data.shape[1]]
    elif filt == "v":
        result = data[startV : startV + data.shape[0],startH + 8//num_bin : startH + data.shape[1]] / flat_im[startV : startV + data.shape[0],startH + 8//num_bin : startH + data.shape[1]]
    else:
        result = data[startV : startV + data.shape[0], startH : startH + data.shape[1]] / flat_im[startV : startV + data.shape[0], startH : startH + data.shape[1]]
    
    # result[np.isinf(result)] = 0
    return result

In [232]:
def DN2ref(filt, data):
    scale_factor = {"ul":6.259, "b":1.254, "v":1, "w":0.645, "x":0.6, "p":1.514, "zs":1}
    result = scale_factor[filt] * data

    return result

In [234]:
def pre_correction(path):            
    hdulist = fits.open(path)
    hdu = hdulist[0]
    data = hdu.data
    header = hdu.header
    date = header["UTC_0"]
    date = re.split('[-T]',date)
    l_year = 2003
    l_month = 5
    l_day = 9
    l_date = datetime(l_year,l_month,l_day)
    year = int(date[0])
    month = int(date[1])
    day = int(date[2])
    date = datetime(year,month,day)
    days = date - l_date
    DAY = days.days
    xs = header["NAXIS1"]
    ys = header["NAXIS2"]
    startH = header["START_H"]
    startV = header["START_V"]
    filt = header["FILTER_0"]
    tEXP = header["EXP_0"]
    num_bin = header["BINNING"]
    status = check_need_smear(path)
    if(status == 0):
        bias = bias_cor(DAY)
        bias_data = data - bias
        data = smear_cor(bias_data, ys, xs, tEXP, bias)
    print(data[117][117])
    flat_data = flat_cor(filt, data, num_bin, startH, startV)
    print(flat_data[117][117])
    scattered_data = scattered_light(flat_data, filt)
    print(scattered_data[117][117])
    # result = DN2ref(filt, scattered_data)

    parts = path.split("st_")
    name = f"cor{status}_st_{parts[1]}"
    print(f"st_{parts[1]}  done")
    hdu = fits.PrimaryHDU(scattered_data)
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(name, overwrite=True)

    return scattered_data
    

    

In [86]:
new_directory = "output_preprocess"
original_directory = "/Users/hattoriakihiro/Documents/after_graduate2/preprocess/output_preprocess"
# ディレクトリを作成（同名のディレクトリが存在する場合、上書き）
new_path = Path(new_directory)
new_path.mkdir(exist_ok=True)

# ディレクトリを格納するリストを作成
directory_list = []

# ベースディレクトリのパスを指定
home_dir = os.path.expanduser("~")
base_directory = os.path.join(home_dir, "Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/")

# ベースディレクトリ内のすべてのディレクトリを取得
for directory in os.listdir(base_directory):
    directory_path = os.path.join(base_directory, directory)
    if os.path.isdir(directory_path):
        # ディレクトリ内のファイルを取得
        files = os.listdir(directory_path)
        # "b", "v", "w", "p" が各ファイル名に含まれるセットを検査
        if all(char in "".join(files) for char in "bvpw"):
            if "2005" in directory:
                directory_list.append(directory)

print(directory_list)
for subdir in directory_list:
    image_files = sorted(glob.glob(os.path.join(base_directory, subdir, '*.fit')), key = natural_keys)
    print(image_files)

os.chdir(original_directory)
for directory in directory_list:
    target_dir = os.path.join(original_directory, directory)
    target_path = Path(target_dir)
    target_path.mkdir(exist_ok=True)
    os.chdir(target_dir)
    # for f in image_files:
    #     pre_correction(f)
    print(os.getcwd())
    os.chdir("..")
    print(os.getcwd())
os.chdir(original_directory)
print(os.getcwd())

['20050928', '20050917', '20050921', '20050919', '20050918', '20051005', '20050920', '20050929', '20051016', '20051011', '20051018', '20050904', '20051019', '20050903', '20051010', '20051017', '20051119', '20050914', '20051009', '20050925', '20050922', '20051006', '20050923', '20050924', '20051008', '20051030', '20050908', '20050906', '20051023', '20051024', '20051025', '20051022', '20051014', '20051013', '20050909', '20051103', '20051104', '20050829']
['/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414830788_v.fit', '/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414838473_v.fit', '/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414846142_v.fit', '/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414853859_v.fit', '/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414861544_v.fit', '/Users/hattoriakihiro/Documents/gra/

In [237]:
test_dir = "../../HAY_A_AMICA_3_HAYAMICA_V1_0/data/"
save_dir = "test"
os.chdir("/Users/hattoriakihiro/Documents/after_graduate2/preprocess/")
path = Path(save_dir)
path.mkdir(exist_ok=True)
os.chdir(save_dir)
image_files = sorted(glob.glob(os.path.join(test_dir, "20051025", '*.fit')), key = natural_keys)
for i in range(len(image_files)):
    fdata = pre_correction(image_files[i])
os.chdir("/Users/hattoriakihiro/Documents/after_graduate2/preprocess/")


2929
2594.5411174436


/var/folders/pz/yjmbqcmx2t5cm5ff1q7nkqnc0000gn/T/ipykernel_93654/2027342670.py:18: RuntimeWarning: divide by zero encountered in true_divide
  result = data[startV : startV + data.shape[0],startH + 70//num_bin : startH + data.shape[1]] / flat_im[startV : startV + data.shape[0],startH + 70//num_bin : startH + data.shape[1]]
/var/folders/pz/yjmbqcmx2t5cm5ff1q7nkqnc0000gn/T/ipykernel_93654/2027342670.py:18: RuntimeWarning: invalid value encountered in true_divide
  result = data[startV : startV + data.shape[0],startH + 70//num_bin : startH + data.shape[1]] / flat_im[startV : startV + data.shape[0],startH + 70//num_bin : startH + data.shape[1]]


nan
st_2489428266_b.fit  done
2843
2595.114154472709


/var/folders/pz/yjmbqcmx2t5cm5ff1q7nkqnc0000gn/T/ipykernel_93654/2027342670.py:20: RuntimeWarning: divide by zero encountered in true_divide
  result = data[startV : startV + data.shape[0],startH + 8//num_bin : startH + data.shape[1]] / flat_im[startV : startV + data.shape[0],startH + 8//num_bin : startH + data.shape[1]]
/var/folders/pz/yjmbqcmx2t5cm5ff1q7nkqnc0000gn/T/ipykernel_93654/2027342670.py:20: RuntimeWarning: invalid value encountered in true_divide
  result = data[startV : startV + data.shape[0],startH + 8//num_bin : startH + data.shape[1]] / flat_im[startV : startV + data.shape[0],startH + 8//num_bin : startH + data.shape[1]]


nan
st_2489432123_v.fit  done
3068
3083.209840490721
2965.411196665661
st_2489436012_w.fit  done
2768
2775.441999542201
2339.9468486495807
st_2489439885_p.fit  done
0
0.0


KeyboardInterrupt: 

In [143]:
len(image_files)

19

In [131]:
print(fdata)

moon = fits.getdata("./moon_cor.fit") #moon
a = pre_correction("../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20040517/st_1035463706_p.fit")
moon[117][117]

[[8.31394863e+02 2.19865740e+02 1.08481913e+02 ... 4.87197007e+01
             inf 1.38613688e+03]
 [0.00000000e+00 6.32745203e+02 3.31730183e+02 ... 9.70000363e+00
             inf 2.59956802e+03]
 [1.11949780e+03 6.24334566e+02 4.25662279e+02 ... 1.95069178e+01
             inf 5.14771408e+03]
 ...
 [2.41777765e+02 2.23649899e+02 9.26829105e+00 ... 2.23461504e+01
             inf            inf]
 [2.43050254e+02 2.37589770e+02 3.70689380e+01 ... 1.14916689e+01
             inf            inf]
 [6.54819895e+02 6.46770767e+02 3.71116382e+01 ... 2.40551022e+01
             inf 1.04207785e+04]]
662.1213367655679
660.0237808927913
578.340492837947


578.340492837947

In [139]:
print((moon == a).all())
0//4

True


0

In [220]:
print(os.getcwd())
save_dir = ("../flat_filt/crop_ver/")
import pandas as pd
flat_files = sorted((glob.glob("../flat_filt/*.fit")), key = natural_keys)
def find_first_zero(row):
    return row.eq(0).idxmax()
for flat_file in flat_files:
    name = flat_file.split("bin")
    num_bin = flat_file.split("n_")
    num_bin = num_bin[1].split("flat")
    num = int(num_bin[0])
    n = f"bin{name[1]}"
    print(n)
    pat = flat_file.split("/bin")
    pa = pat[1].split("t_")
    p = pa[1].split(".f")
    filt = p[0]
    print(filt)
    data = fits.getdata(flat_file)
    contains_zero = (data == 0).any()
    if contains_zero:
        
        df = pd.DataFrame(data)
        if filt == "b":
            a = df.iloc[0//num:1022//num,70//num:1014//num]
        elif filt == "v":
            a = df.iloc[0//num:1022//num,8//num:1014//num]
        else:
            a = df.iloc[0//num:1022//num,:]
        rows_with_zeros = a.index[a.apply(lambda row: any(row == 0), axis=1)].tolist()
        columns_with_zeros = a.columns[a.isin([0]).any()].tolist()
        print("0を含む行が何行目か:", rows_with_zeros)
        print("0を含む列が何列目か:", columns_with_zeros)
        data = a.values
        fits.writeto(f"{save_dir}{n}",data, overwrite = True)
    else:
        fits.writeto(f"{save_dir}{n}",data, overwrite = True)

    # 各行で最初に0を持つ列の位置を見つ



/Users/hattoriakihiro/Documents/after_graduate2/preprocess/test
bin_1flat_b.fit
b
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_1flat_p.fit
p
bin_1flat_ul.fit
ul
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_1flat_v.fit
v
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_1flat_w.fit
w
bin_1flat_wide.fit
wide
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_1flat_x.fit
x
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_1flat_zs.fit
zs
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_2flat_b.fit
b
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_2flat_p.fit
p
bin_2flat_ul.fit
ul
bin_2flat_v.fit
v
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_2flat_w.fit
w
bin_2flat_wide.fit
wide
bin_2flat_x.fit
x
bin_2flat_zs.fit
zs
bin_4flat_b.fit
b
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_4flat_p.fit
p
bin_4flat_ul.fit
ul
bin_4flat_v.fit
v
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_4flat_w.fit
w
bin_4flat_wide.fit
wide
bin_4flat_x.fit
x
bin_4flat_zs.fit
zs
bin_8flat_b.fit
b
0を含む行が何行目か: []
0を含む列が何列目か: []
bin_8flat_p.fit
p
bin_8flat_ul.fit
ul
bin_8flat_v.fit
v
bin_8flat_w.fit
w
bin_8flat_wide.fit
wide
bin_8flat_x.fit
x
bin_8flat_

In [216]:
data = fits.getdata("../flat_filt/bin_1flat_ul.fit")
df = pd.DataFrame(data)
# a = df.iloc[0:1022,70:1014]
a = df
rows_with_zeros = a.index[a.apply(lambda row: any(row == 0), axis=1)].tolist()
columns_with_zeros = a.columns[a.isin([0]).any()].tolist()
name = flat_file.split("bin")
n = f"bin{name[1]}"
print("0を含む行が何行目か:", rows_with_zeros)
print("0を含む列が何列目か:", columns_with_zeros)
a

0を含む行が何行目か: [1023]
0を含む列が何列目か: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.088363,0.076548,0.078323,0.082077,0.081335,0.085111,0.085846,0.091120,0.097915,0.111184,...,0.136553,0.119538,0.109730,0.104441,0.102193,0.102119,0.099901,0.100890,0.106209,0.120984
1,0.091607,0.080826,0.081081,0.083089,0.080302,0.081095,0.086894,0.091113,0.095675,0.108952,...,0.138590,0.120534,0.111521,0.105954,0.102455,0.100388,0.101152,0.104164,0.106194,0.120992
2,0.094139,0.083613,0.080564,0.082849,0.083830,0.084340,0.089389,0.090626,0.095406,0.109221,...,0.140336,0.123044,0.110015,0.104449,0.100950,0.100148,0.102658,0.105663,0.108937,0.122992
3,0.096140,0.083598,0.083343,0.081597,0.082572,0.085104,0.088385,0.090378,0.098148,0.109723,...,0.140801,0.124295,0.112772,0.099676,0.094671,0.097151,0.104906,0.104681,0.106180,0.121486
4,0.096162,0.083089,0.078593,0.078578,0.078301,0.083096,0.086370,0.091630,0.097399,0.111716,...,0.141303,0.123524,0.112510,0.101437,0.098417,0.098140,0.104149,0.103131,0.104681,0.122483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0.123014,0.109933,0.107169,0.106427,0.109941,0.107161,0.109955,0.116751,0.128319,0.145116,...,0.111709,0.095856,0.082834,0.081073,0.073813,0.074083,0.074322,0.072547,0.077305,0.091622
1020,0.121748,0.109184,0.107940,0.107183,0.107693,0.110435,0.111229,0.121741,0.130537,0.147379,...,0.111709,0.093121,0.086857,0.081058,0.075566,0.072322,0.075087,0.076615,0.077552,0.095383
1021,0.119980,0.107708,0.108427,0.105685,0.108457,0.109461,0.110173,0.116998,0.125240,0.144869,...,0.111207,0.093855,0.086333,0.083089,0.074046,0.073333,0.074330,0.076600,0.079065,0.091353
1022,0.117216,0.106442,0.103445,0.103684,0.105932,0.105445,0.107446,0.115500,0.122752,0.142554,...,0.106187,0.090851,0.085104,0.080317,0.074023,0.074810,0.076076,0.075326,0.075042,0.091135


In [206]:
data = fits.getdata("../flat_filt/bin_1flat_v.fit")
path = "../flat_filt/bin_1flat_v.fit"
pat = path.split("/bin")
pa = pat[1].split("t_")
p = pa[1].split(".f")
filt = p[0]
print(filt)

df = pd.DataFrame(data)
df
a = df.iloc[0:1022,8:1014]
a
rows_with_zeros = a.index[a.apply(lambda row: any(row == 0), axis=1)].tolist()
columns_with_zeros = a.columns[a.isin([0]).any()].tolist()
print("0を含む行が何行目か:", rows_with_zeros)
print("0を含む列が何列目か:", columns_with_zeros)

v
0を含む行が何行目か: []
0を含む列が何列目か: []


In [223]:
data.shape

(128, 128)